# Smooth Cubic Multivariate Local Interpolations

* Cubic grid interpolation scheme in up to 4 dimensions
* Uses only local data, i.e. grid points touching a cell
* Uses values and all combinations of first-order derivatives, e.g. in 2D f_x, f_y, f_xy
* Based on Lekien & Marsden 2005:
   * Only 3D
   * Coefficients calculated by Matrix inversion (slow)
* Our improvements:
   * Added 1D to 4D
   * Coefficients directly calculated by Maple-generated C-code

## `LocalInterpolations.jl`

* Julia package

## `loci`

* Library written in C
* Static library for C/C++
* Shared library for Python and others (e.g. FORTRAN)

## Python usage

### import numpy, scipy and loci

In [ ]:
from numpy import *

from loci import Interpolation, Range

### import libraries for example

In [ ]:
%matplotlib inline 
from matplotlib.pyplot import *
from scipy import *

rcParams['figure.figsize'] = (10, 6)
rcParams.update({'font.size': 16})

#Plotting Colors
lin  = ["#000000", "#FF0000", "#0000FF", "#FFA500", "#008000", "#808080", 
        "#800080", "#008080"]

### Define functions and derivatives

In [ ]:
A=2.
B=0.5
def f(x, y):    return log(A*x**2 + B*y**2 + 1)
def f_x(x, y):  return 2*A*x/(A*x**2 + B*y**2 + 1)
def f_y(x, y):  return 2*B*y/(A*x**2 + B*y**2 + 1)
def f_xy(x, y): return -4*A*B*x*y/(A*x**2 + B*y**2 + 1)**2

### Define interpolation ranges

In [ ]:
rx  = Range(1., 0.1, 10)    #x0 =1., dx=0.1, lenght=
ry  = Range(-2., 0.5, 20)

rx = $1.0, 1.1, ..., 1.9$

#### Last element of `rx`
   `rx.x0 + (rx.len - 1)*rx.dx`

### Create interpolation

In [ ]:
ip  = Interpolation(rx, ry, f, f_x, f_y, f_xy)

`ip` is defined on Range from `rx.x0` to `(rx.len - 2)*rx.dx`. Last gridpoint of rx out of bounds!

### Interpolate at a given point

In [ ]:
xt  = rx.x0 + rand()*(rx.len - 2)*rx.dx
yt  = ry.x0 + rand()*(ry.len - 2)*ry.dx

f0  = f(xt, yt)
fi  = ip.interpolate(xt, yt)

d   = fi/f0 - 1.

print " f(%f, %f)\t= %0.8e"%(xt, yt, f0)
print "ip(%f, %f)\t= %0.8e"%(xt, yt, fi)
print "diff\t\t\t= %0.8e"%d

### Interpolated derivatives in x and y

In [ ]:
d_x = ip.diff_x(xt, yt)/f_x(xt, yt) - 1
d_y = ip.diff_y(xt, yt)/f_y(xt, yt) - 1

print "diff_x =  %e\ndiff_y = %e"%(d_x, d_y)

### Interpolated 2nd-order x and 3rd-order y derivative

In [ ]:
print ip.diff(1, 0, xt, yt)

### Interpolate out of bounds

In [ ]:
print ip.interpolate(rx.x0 - 1, ry.x0)
print ip.interpolate(rx.x0, ry.x0 + (ry.len-1)*ry.dx)

### Create points in range rx for mapping

In [ ]:
xs  = r_[rx.x0:rx.x0+(rx.len - 1)*rx.dx:rx.dx/1000]
ys  = zeros(len(xs))

fs  = f(xs, ys)
ips = ip.map(xs, ys)

### Map interpolation and derivatives in x, y and 2nd-order x and 3rd-order y on points

In [ ]:
plot(xs, fs, "-", color=lin[3], linewidth=3, label="$f(x, y=0)$")
plot(xs, ips, "-", color=lin[0], label="ip$(x, y=0)$")
plot(xs, ip.map_x(xs, ys), "-", color=lin[1], label="ip$_x(x, y=0)$")
plot(xs, ip.map_y(xs, ys), "-", color=lin[2], label="ip$_y(x, y=0)$")
plot(xs, ip.map_diff(2, 3, xs, ys), "-", color=lin[4], label="ip$_{xx,yyy}(x, y=0)$")

legend(loc=0, fontsize=12)
xlabel("$x$")

In [ ]:
semilogy(xs, abs(fs/ips - 1), "-", color=lin[0])

xlabel("$x$")
ylabel("$f/\mathrm{ip}-1$")